In [1]:
import csv
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from requests_html import HTMLSession
import nest_asyncio

In [2]:
frameworks = ['Frontend','Mobile','Backend','Software-Engineering/OOP','Data Science', 'Low Level',
              'Databases','Cloud Computing','AI / ML','Testing/Utility', 'Animation']

skill_lst_pattern = ['Java','C#','Python','ReactJS','VueJS','AngularJS','React','Vue','Angular','CSS','HTML','spring'
              'nodeJS',' JS','PHP','node.js','NodeJS','mongoDB','MongoDB','SQL','NoSQL','ARM','Assembly','Arduino',
              'Raspberry Pi','Pi','GraphQL','Aws','AWS','Cloud','Automation','Testing','Unit Testing','spring','Spring',
              'Integration Testing','CMS','Maven','Kotlin','Headless','iOS','IOS','Android','Flutter','Linux','Server',
              'Data Processing','Data Cleaning','Data Mining','Data Science','Docker','PostgreSQL','Typescript','Restful',
              'API','Ruby','Ruby on Rails','ES6','ES5','Json','Android Studio','React Native','Azure','UI/UX','UI','UX',
              'Gatsby','Go','Golang','Dart','Javascript','JavaScript','Cuda','Unix','Gpu','GPU','Big Data','ASP',
              'Asp.net','Objective-C','objective-C','Objective C','Animation','Graphics','Perl','Laravel','XML','Swift','Scala',
              'Rust','Jquery','JQuery','Bootstrap','Git','Github','MVC','AI','ML','Machine Learning','Artificial Intelligence',
              'Django','Flask','Express','Express.js','ExpressJS','OOP','Raspberry','PI','Database','postgreSQL','mySQL','MySQL',
              'SQlite','SQLite','Oracle','oracle','firebase','Firebase','cloud','Cloud Computing','TensorFlow','Tensorflow','NLP',
              'Natural Language Processing','Prolog','OS','Operating System']


skill_dict = {} #Initialize our dictionary with frameworks as keys and empty lists as values
job_desc = [] #list containing all descriptions

for i in frameworks:
    skill_dict[i] = []



## Generate a url with our input parameters

In [3]:
def get_url(position,location,date):
    
    template = 'https://www.indeed.com/jobs?q={}&l={}&fromage={}'
    url = template.format(position,location,date)
    return url

## Generate full description url from first url

In [4]:
def get_details_url(url,advn,vjk):
    
    template = url + '&advn={}&vjk={}'
    details_url = template.format(advn,vjk)
    return details_url

## Clean and process Data

In [5]:
def process_text(desc,card):
    
    cleantxt = ''.join(map(str,desc)) # Turn list into string

    ## Find skills
    match1 = (re.findall(r"(?=("+'|'.join(skill_lst_pattern)+r"))", cleantxt)) # Matches our list of skills
    match2 = (re.findall("\ .Net|C\/C\+\+|\ .net|node\.js|Node\.js|ASP\.Net |C\+\+|C\ ", cleantxt)) # Matches C/C++ and C and .net etc ...
    skill_lst = match1+match2 # Final result of both our regex matches
    return skill_lst
     

In [6]:
# def fill_dict(index):
    
#     if(index=='UI/UX'or index=='UI'or index=='UX' or index=='ReactJS'or index=='VueJS'or index=='AngularJS'or index=='React'or index=='Vue'|
#        index=='Angular'|index=='CSS'|index=='HTML'|index=='JS'|index=='Typescript'|index=='Json'|index=='ES5'|index=='ES6'|
#        index=='Gatsby'|index=='Javascript'|index=='JavaScript'|index=='Bootstrap'|index=='JQuery'|index=='Jquery'):
#         skill_dict['Frontend'] = index
        
#     if(index=='Java'|index=='iOS'|index=='IOS'|index=='Android'|index=='Flutter'|index=='Android Studio'|index=='React Native'|
#        index=='Swift'|index=='Dart'|index=='Kotlin'):
#         skill_dict['Mobile'] = index
        
#     if(index=='nodeJS'|index=='node.js'|index=='NodeJS'|index=='Django'|index=='Flask'|index=='PHP'|index=='GraphQL'|
#        index=='Express'|index=='Express.js'|index=='ExpressJS'|index=='Laravel'|index=='Ruby on Rails'|index=='Ruby'|
#        index=='MVC'|index=='Spring'|index=='spring'|index=='server'|index=='ASP'|index=='ASP.Net'|index=='.Net'|index=='.net'):
#         skill_dict['Backend'] = index
        
#     if(index=='Java'|index=='C'|index=='C/C++'|index=='C#'|index=='Golang'|index=='Go'|index=='objective-C'|index=='Objective-C'|
#        index=='Objective C'|index=='Scala'|index=='Rust'|index=='Python'|index=='OOP'):
#         skill_dict['Software-Engineering/OOP'] = index
        
#     if(index=='Python'|index=='Perl'|index=='Big Data'|index=='Data Mining'|index=='Data Science'|index=='Data Processing'|
#        index=='Data Cleaning'):
#         skill_dict['Data Science'] = index
        
#     if(index=='C'|index=='Arm'|index=='Assembly'|index=='Go'|index=='Golang'|index=='Arduino'|index=='PI'|index=='Raspberry Pi'|
#        index=='Raspberry'|index=='Pi'|index=='Linux'|index=='Unix'|index=='Objective C'|index=='Objective-C'|index=='objective-C'|
#        index=='C/C++'):
#         skill_dict['Low Level'] = index
        
#     if(index=='SQL'|index=='mongoDB'|index=='MongoDB'|index=='Database'|index=='PostgreSQL'|index=='postgreSQL'|index=='mySQL'|
#        index=='MySQL'|index=='SQlite'|index=='SQLite'|index=='NoSQL'|index=='Oracle'|index=='oracle'|index=='XML'|index=='firebase'|
#       index=='Firebase'):
#         skill_dict['Databases'] = index
        
#     if(index=='AWS'|index=='Azure'|index=='Go'|index=='Golang'|index=='cloud'|index=='Cloud Computing'|index=='Cloud'|
#        index=='Automation'|index=='Amazon Web Services'):
#         skill_dict['Cloud Computing'] = index
        
#     if(index=='Python'|index=='TensorFlow'|index=='Tensorflow'|index=='NLP'|index=='Natural Language Processing'|index=='Prolog'|
#        index=='Machine Learning'|index=='Artificial Intelligence'|index=='AI'|index=='ML'):
#         skill_dict['AI / ML'] = index
        
#     if(index=='Git'|index=='Github'|index=='OS'|index=='Operating System'|index=='Docker'|index=='API'|index=='CMS'|index=='Headless'):
#         skill_dict['Testing/Utility'] = index
        
#     if(index=='GPU'|index=='Cuda'|index=='Animation'|index=='Graphics'):
#         skill_dict['Animation'] = index

In [7]:
def fill_dict(index):
    
    if(index=='UI/UX'or index=='UI'or index=='UX' or index=='ReactJS'or index=='VueJS'or index=='AngularJS'or index=='React'or index=='Vue'or 
       index=='Angular'or index=='CSS'or index=='HTML'or index=='JS'or index=='Typescript'or index=='Json'or index=='ES5'or index=='ES6'or 
       index=='Gatsby'or index=='Javascript'or index=='JavaScript'or index=='Bootstrap'or index=='JQuery'or index=='Jquery'):
        skill_dict['Frontend'].append(index)
        
    if(index=='Java'or index=='iOS'or index=='IOS'or index=='Android'or index=='Flutter'or index=='Android Studio'or index=='React Native'or 
       index=='Swift'or index=='Dart'or index=='Kotlin'):
        skill_dict['Mobile'].append(index)
        
    if(index=='nodeJS'or index=='node.js'or index=='NodeJS'or index=='Django'or index=='Flask'or index=='PHP'or index=='GraphQL'or 
       index=='Express'or index=='Express.js'or index=='ExpressJS'or index=='Laravel'or index=='Ruby on Rails'or index=='Ruby'or 
       index=='MVC'or index=='Spring'or index=='spring'or index=='server'or index=='ASP'or index=='ASP.Net'or index=='.Net'or index=='.net'):
        skill_dict['Backend'].append(index)
        
    if(index=='Java'or index=='C'or index=='C/C++'or index=='C#'or index=='Golang'or index=='Go'or index=='objective-C'or index=='Objective-C'or 
       index=='Objective C'or index=='Scala'or index=='Rust'or index=='Python'or index=='OOP'):
        skill_dict['Software-Engineering/OOP'].append(index)
        
    if(index=='Python'or index=='Perl'or index=='Big Data'or index=='Data Mining'or index=='Data Science'or index=='Data Processing'or 
       index=='Data Cleaning'):
        skill_dict['Data Science'].append(index)
        
    if(index=='C'or index=='Arm'or index=='Assembly'or index=='Go'or index=='Golang'or index=='Arduino'or index=='PI'or index=='Raspberry Pi'or 
       index=='Raspberry'or index=='Pi'or index=='Linux'or index=='Unix'or index=='Objective C'or index=='Objective-C'or index=='objective-C'or 
       index=='C/C++'):
        skill_dict['Low Level'].append(index)
        
    if(index=='SQL'or index=='mongoDB'or index=='MongoDB'or index=='Database'or index=='PostgreSQL'or index=='postgreSQL'or index=='mySQL'or 
       index=='MySQL'or index=='SQlite'or index=='SQLite'or index=='NoSQL'or index=='Oracle'or index=='oracle'or index=='XML'or index=='firebase'or 
      index=='Firebase'):
        skill_dict['Databases'].append(index)
        
    if(index=='AWS'or index=='Azure'or index=='Go'or index=='Golang'or index=='cloud'or index=='Cloud Computing'or index=='Cloud'or 
       index=='Automation'or index=='Amazon Web Services'):
        skill_dict['Cloud Computing'].append(index)
        
    if(index=='Python'or index=='TensorFlow'or index=='Tensorflow'or index=='NLP'or index=='Natural Language Processing'or index=='Prolog'or 
       index=='Machine Learning'or index=='Artificial Intelligence'or index=='AI'or index=='ML'):
        skill_dict['AI / ML'].append(index)
        
    if(index=='Git'or index=='Github'or index=='OS'or index=='Operating System'or index=='Docker'or index=='API'or index=='CMS'or index=='Headless'):
        skill_dict['Testing/Utility'].append(index)
        
    if(index=='GPU'or index=='Cuda'or index=='Animation'or index=='Graphics'):
        skill_dict['Animation'].append(index)

In [8]:
# def main (position, location, date, count):
    
#     url = get_url(position,location,date)

#     while count>0:
#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, 'html.parser')
#         cards = soup.find_all('div', 'jobsearch-SerpJobCard')
#         length = len(cards)
        
#         for x in range(5):
#             advn = cards[x].get('data-empn')
#             vjk = cards[x].get('data-jk')
#             details_url = get_details_url(url,advn,vjk) # Generate Url containing full description of jobs

#             #Open and load page using webdriver so we can get parse elements that load after our initial request

#             browser = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')  
#             browser.get(details_url)
#             htmlSource = browser.page_source
#             txt = BeautifulSoup(htmlSource, 'html.parser')
#             browser.close()
# #             job_desc.append(txt.find('div','jobsearch-JobComponent'))  #Append the full description of the job to our list
#             job_desc.append(txt.find('div','jobsearch-jobDescriptionText'))  #Append the full description of the job to our list
#             processed_txt = process_text(job_desc,cards[x])
#             print(details_url)

#         try:
#             url = 'https://www.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
#         except AttributeError:
#             break
#         count-= 1
        
#         ## Fill framework list
#         index = len(processed_txt)
#         for i in range(index):
#             fill_dict(processed_txt[i])
#     print(skill_dict['Frontend'])


In [9]:
count = 1

url = get_url('Software Engineer','Los Angeles',3)

while count>0:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')
    length = len(cards)
        
    for x in range(5):
        advn = cards[x].get('data-empn')
        vjk = cards[x].get('data-jk')
        details_url = get_details_url(url,advn,vjk) # Generate Url containing full description of jobs

            #Open and load page using webdriver so we can get parse elements that load after our initial request

        browser = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')  
        browser.get(details_url)
        htmlSource = browser.page_source
        txt = BeautifulSoup(htmlSource, 'html.parser')
        browser.close()
#            job_desc.append(txt.find('div','jobsearch-JobComponent'))  #Append the full description of the job to our list
        job_desc.append(txt.find('div','jobsearch-jobDescriptionText'))  #Append the full description of the job to our list
        processed_txt = process_text(job_desc,cards[x])
        print(details_url)

    try:
        url = 'https://www.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
    except AttributeError:
        break
    count-= 1
        
    ## Fill framework list
    index = len(processed_txt)
    for i in range(index):
        fill_dict(processed_txt[i])

https://www.indeed.com/jobs?q=Software Engineer&l=Los Angeles&fromage=3&advn=2885959151936949&vjk=d15ed09ac459fe7e
https://www.indeed.com/jobs?q=Software Engineer&l=Los Angeles&fromage=3&advn=4407846845847026&vjk=d0110cf4d5d93716
https://www.indeed.com/jobs?q=Software Engineer&l=Los Angeles&fromage=3&advn=6759004179576592&vjk=a39920f36d41e3af
https://www.indeed.com/jobs?q=Software Engineer&l=Los Angeles&fromage=3&advn=None&vjk=839a5426d06075d2
https://www.indeed.com/jobs?q=Software Engineer&l=Los Angeles&fromage=3&advn=None&vjk=6c66fea827538a17


In [10]:
# main('Software Engineer', 'Los Angeles', 3 , 1)
print(skill_dict['Frontend'])
print(skill_dict['Backend'])
print(skill_dict['Software-Engineering/OOP'])
print(skill_dict['Data Science'])
print(skill_dict['Low Level'])
print(skill_dict['Mobile'])
print(skill_dict['AI / ML'])
print(skill_dict['Databases'])
print(skill_dict['Testing/Utility'])
print(skill_dict['Animation'])
print(skill_dict['Cloud Computing'])



['React', 'Angular', 'React', 'HTML', 'CSS', 'React', 'UI/UX', 'UX']
['Spring', 'Spring', 'NodeJS']
['Java', 'Java', 'Java', 'C#']
[]
['Linux', 'Linux']
['Java', 'Java', 'Java']
['ML', 'ML']
['XML', 'SQL']
['Git', 'OS', 'OS', 'Git']
[]
['cloud', 'AWS', 'AWS']


In [11]:
# get salary
# get date
# to csv
# to firebase

In [12]:
    ## Find salary TODO
#     try:
#         job_salary = card.find('span','salaryText').text.strip()
#     except AttributeError:
#         job_salary = ''
#     print(job_salary)
    
#     ## Find date
# #     job_date = card.find('span','date').text
# #     print(job_date)
#     try:
#         job_date_text = match.expand(re.findall("\d day ago|\d\d days ago| just now| today ", cleantxt))
#     #     job_date_digit = (re.findall("\d\d|\d", job_date_text))
#         today = datetime.today().strftime('%Y-%m-%d')
#         print(today)
#         print(job_date_text)
#     except AttributeError:
#         job_date_text = ''  
    
    ## Find experience ## TODO

In [13]:
# ##### Loop thru all the jobs and gather descriptions

# job_desc = [] #list containing all descriptions
# length = len(cards)
# for x in range(length):
#     advn = cards[x].get('data-empn')
#     vjk = cards[x].get('data-jk')
#     details_url = get_details_url(url,advn,vjk) #append and generate new Url

#     #Open and load page using webdriver so we can get parse elements that load after our initial request
    
#     browser = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')  
#     browser.get(details_url)
#     htmlSource = browser.page_source
#     txt = BeautifulSoup(htmlSource, 'html.parser')
#     browser.close()
#     job_desc.append(txt.find('div','jobsearch-jobDescriptionText'))  #Append the full description of the job to our list
#     job_desc.append('#---------------------------------------------------------------------------------------------------#')
#     print(details_url)
#     job_desc.append('#---------------------------------------------------------------------------------------------------#')
        
# # print(job_desc) 
    

In [14]:
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import PlaintextCorpusReader
# string1 = ''.join(map(str,job_desc))   
# cleanText = nltk.Text(nltk.word_tokenize(string1))
# # corp = PlaintextCorpusReader(job_desc)
# # cleanText = nltk.Text(corp.words())
# match = cleanText.concordance('Java')
# match2 = cleanText.concordance('ReactJS')
# string2 = ''.join(map(str,job_desc))
# print(match)
# print(match2)
# if 'at' in listOfStrings :

In [15]:

# cleantxt = ''.join(map(str,job_desc))
# x = "s C++ C/C++ and .net and .Net and node.js and Java and UI/UX and C and C++ and C/C++s and node.js and . and c capital"
# match1 = (re.findall(r"(?=("+'|'.join(skill_lst_pattern)+r"))", cleantxt)) # Matches our list of skills
# match2 = (re.findall("\.Net|C\/C\+\+|\.net|node\.js|Node\.js|ASP\.Net |C\+\+|C\ ", cleantxt)) # Matches C/C++ and C and .net etc ...
# skill_lst = match1+match2 # Final result of both our regex matches
# # print(skill_lst)
skill_lst_pattern = ['Java','C#','Python','ReactJS','VueJS','AngularJS','React','Vue','Angular','CSS','HTML','spring'
              'nodeJS',' JS','PHP','node.js','NodeJS','mongoDB','MongoDB','SQL','NoSQL','ARM','Assembly','Arduino',
              'Raspberry Pi','Pi','GraphQL','Aws','AWS','Cloud','Automation','Testing','Unit Testing','spring','Spring',
              'Integration Testing','CMS','Maven','Kotlin','Headless','iOS','IOS','Android','Flutter','Linux','Server',
              'Data Processing','Data Cleaning','Data Mining','Data Science','Docker','PostgreSQL','Typescript','Restful',
              'API','Ruby','Ruby on Rails','ES6','ES5','Json','Android Studio','React Native','Azure','UI/UX','UI','UX',
              'Gatsby','Go','Golang','Dart','Javascript','JavaScript','Cuda','Unix','Gpu','GPU','Big Data','ASP',
              'Asp.net','Objective-C','objective-C','Objective C','Animation','Graphics','Perl','Laravel','XML','Swift','Scala',
              'Rust','Jquery','JQuery','Bootstrap','Git','Github','MVC','AI','ML','Machine Learning','Artificial Intelligence'
              ]

In [16]:
count = 0
num = skill_lst[0]
counter = 0
for i in skill_lst:
#     if(i == 'JS' or i == 'React' or i == 'Angular' or i == 'Vue' or i == 'Node.js'):
    if(i == 'Java'):
        counter+=1
    curr_freq = skill_lst.count(i)
    if(curr_freq> count):
        count = curr_freq
        num = i

print(num)
print(counter)
len(skill_lst)

NameError: name 'skill_lst' is not defined